In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
!pip install -q peft
!pip install -q accelerate
!pip install -U bitsandbytes
!pip install -q transformers
!pip install -q datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 34.1 MB/s eta 0:00:00:00:0100:01


In [2]:
!pip install GPUtil

  Preparing metadata (setup.py) ... done
  Created wheel for GPUtil: filename=GPUtil-1.4.0-py3-none-any.whl size=7392 sha256=c146cd718762ea5737f85fad55eb72bc0bebd25d64bee23b3bd63961ea913008
  Stored in directory: /root/.cache/pip/wheels/92/a8/b7/d8a067c31a74de9ca252bbe53dea5f896faabd25d55f541037
Successfully built GPUtil


In [3]:
import torch
import GPUtil
import os

GPUtil.showUtilization()

if torch.cuda.is_available():
    print("GPU is available")
else:
    print("GPU is not available, using CPU instead")

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

| ID | GPU | MEM |
------------------
|  0 |  0% |  0% |
|  1 |  0% |  0% |
GPU is available


In [4]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, LlamaTokenizer
from huggingface_hub import notebook_login
#from datasets import load_dataset
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model

if "COLAB_GPU" in os.environ:
  from google.colab import output
  output.enable_custom_widget_manager()


2026-01-22 16:27:33.657523: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1769099253.806480      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1769099253.851996      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1769099254.205038      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1769099254.205075      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1769099254.205078      55 computation_placer.cc:177] computation placer alr

In [5]:
if "COLAB_GPU" in os.environ:
  !huggingface-cli login
else:
  notebook_login()

In [6]:
base_model_id = "meta-llama/Llama-3.2-3B-Instruct"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(base_model_id, quantization_config=bnb_config,device_map="auto")

config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

In [7]:
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

config = LoraConfig(
    r=64,
    lora_alpha=64,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    bias="none",
    lora_dropout=0.05,
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)

In [8]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-3B-Instruct")

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [10]:
if tokenizer.pad_token is None:
  tokenizer.add_special_tokens({'pad_token': tokenizer.eos_token})

In [11]:
import pandas as pd
python_df = pd.read_json(path_or_buf= '/kaggle/input/mbppjsonl/mbpp.jsonl', lines=True)
python_df = python_df[['text', 'code']]
python_df.rename(columns = {'text': 'question', 'code': 'answer'}, inplace=True)
python_df.sample(5)

,question,answer
444,Write a function to perform index wise multipl...,"def index_multiplication(test_tup1, test_tup2)..."
251,Write a python function to convert complex num...,import cmath \r\ndef convert(numbers): \r\...
406,Write a function to create the next bigger num...,def rearrange_bigger(n):\r\n nums = list(st...
294,Write a function to return the sum of all divi...,def sum_div(number):\r\n divisors = [1]\r\n...
667,Write a python function to replace multiple oc...,"import re \r\ndef replace(string, char): \r\n ..."


In [12]:
import json
import re
with open('/kaggle/input/kaggle-data-1/kaggle_data.json','r') as f:
    kaggle_data = json.load(f)

In [13]:
kaggle_df = pd.DataFrame(kaggle_data)
def clean_text(text: str) -> str:
    text = re.sub(r'<[^>]+>', '', text) # remove HTML/Markdown tags
    text = re.sub(r'@\w+', '', text) # remove @user tags
    text = text.replace('\n', ' ') # remove newline characters
    text = re.sub(r'\s+', ' ', text) # remove multiple spaces
    text = text.strip() # remove leading and trailing spaces
    return text

kaggle_df['question'] = kaggle_df['question'].apply(clean_text)
kaggle_df['answer'] = kaggle_df['answer'].apply(clean_text)

In [14]:
data_science_df = pd.read_csv('/kaggle/input/data-science-1/DataScience QA.csv')
data_science_df.rename(columns = {'Question': 'question', 'Answer': 'answer'}, inplace=True)

In [15]:
kaggle_df = pd.concat([kaggle_df, python_df, data_science_df])

In [16]:
def text_limit(text: str) -> str:
    max_len = 1024
    return text[:max_len] if len(text) > max_len else text

kaggle_df['question'] = kaggle_df['question'].apply(text_limit)
kaggle_df['answer'] = kaggle_df['answer'].apply(text_limit)

kaggle_df.sample(5)

,question,answer
359,Write a function to find the n’th carol number.,def get_carol(n): \r\n\tresult = (2**n) - 1\r\...
398,Getting HTML elements via XPath in bash,Getting HTML elements via XPath in bash from h...
432,Write a function to check whether the entered ...,"def check_greater(arr, number):\r\n arr.sort(..."
100,Write a function to find the kth element in th...,"def kth_element(arr, n, k):\r\n for i in rang..."
342,Write a function to calculate the number of di...,def dig_let(s):\r\n d=l=0\r\n for c in s:\r\n ...


In [17]:
kaggle_data = []

for _, row in kaggle_df.iterrows():
    kaggle_data.append(f'Question:\n{row["question"]}\n\Answer:\n{row["answer"]}')

kaggle_data[0]

<>:4: SyntaxWarning: invalid escape sequence '\A'
<>:4: SyntaxWarning: invalid escape sequence '\A'
/tmp/ipykernel_55/501628493.py:4: SyntaxWarning: invalid escape sequence '\A'
  kaggle_data.append(f'Question:\n{row["question"]}\n\Answer:\n{row["answer"]}')


"Question:\nCreate a set from a series in pandas\n\\Answer:\nIf you only need to get list of unique values, you can just use unique method. If you want to have Python's set, then do set(some_series) In [1]: s = pd.Series([1, 2, 3, 1, 1, 4]) In [2]: s.unique() Out[2]: array([1, 2, 3, 4]) In [3]: set(s) Out[3]: {1, 2, 3, 4} However, if you have DataFrame, just select series out of it ( some_data_frame[''] )."

In [17]:
tokenized_train_dataset = []
for phrase in kaggle_data:
  tokenized_train_dataset.append(tokenizer(phrase))

In [18]:
LEARNING_RATE = 5e-5
WEIGHT_DECAY = 0.01
BETA_1 = 0.9
BETA_2 = 0.999
trainer = transformers.Trainer(
    model=model,
    train_dataset=tokenized_train_dataset,
    args=transformers.TrainingArguments(
        output_dir="./finetunedModelllama",
        per_device_train_batch_size=2,
        gradient_accumulation_steps=2,
        num_train_epochs=8,
        learning_rate=LEARNING_RATE,
        weight_decay=WEIGHT_DECAY,
        adam_beta1=BETA_1,        
        adam_beta2=BETA_2,
        bf16=False,
        optim="paged_adamw_8bit",
        logging_dir="./log",
        save_strategy="epoch",
        save_steps=50,
        logging_steps=50,
        report_to="none"

),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache=False
trainer.train()

/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:929: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
50,2.020500
100,1.820300
150,1.724100
200,1.810300
250,1.731300
300,1.580300
350,1.709500
400,1.607100
450,1.679200
500,1.724200


/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:929: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:929: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.12/dist-packages/torch/_dynamo/

TrainOutput(global_step=3912, training_loss=0.9184143191588193, metrics={'train_runtime': 26575.4377, 'train_samples_per_second': 0.588, 'train_steps_per_second': 0.147, 'total_flos': 3.672695632342426e+16, 'train_loss': 0.9184143191588193, 'epoch': 8.0})

In [19]:
import os
import shutil
from pathlib import Path

# --- CONFIGURATION ---
# We assume this is the Teacher model based on your previous messages
repo_id = "Rajesh17092004/Kaggle-Assistant-Pro-Llama-3.2-3B"
clean_output_dir = "./final_clean_model"

print("⏳ Starting Export Process...")

# ---------------------------------------------------------
# STEP 1: Save a "Clean" Version (Weights Only)
# ---------------------------------------------------------
# This strips away the massive optimizer states, reducing size by ~70%
print(f"💾 Saving clean model to: {clean_output_dir}...")

# We use the 'trainer' object from your previous cell
trainer.save_model(clean_output_dir)
tokenizer.save_pretrained(clean_output_dir)

# ---------------------------------------------------------
# STEP 2: Push to Hugging Face Hub
# ---------------------------------------------------------
print(f"🚀 Pushing to Hugging Face Hub: {repo_id}...")

try:
    # Push the model and tokenizer we just saved
    trainer.push_to_hub(repo_id)
    tokenizer.push_to_hub(repo_id)
    print("✅ Successfully pushed to Hugging Face!")
except Exception as e:
    print(f"❌ Error pushing to Hub: {e}")
    print("Don't worry, we are still creating the Zip file below!")

# ---------------------------------------------------------
# STEP 3: Zip the CLEAN folder for Local Download
# ---------------------------------------------------------
print("📦 Zipping for local download...")

zip_path = "/kaggle/working/finetuned_teacher_final"
shutil.make_archive(zip_path, 'zip', root_dir=clean_output_dir)

# Print final status
zip_size_mb = os.path.getsize(zip_path + ".zip") / (1024 * 1024)
print(f"\n🎉 ALL DONE!")
print(f"1. Model is live at: https://huggingface.co/{repo_id}")
print(f"2. Zip file is ready: {zip_path}.zip")
print(f"3. Zip Size: {zip_size_mb:.2f} MB")

⏳ Starting Export Process...
💾 Saving clean model to: ./final_clean_model...
🚀 Pushing to Hugging Face Hub: Rajesh17092004/Kaggle-Assistant-Pro-Llama-3.2-3B...


Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

✅ Successfully pushed to Hugging Face!
📦 Zipping for local download...

🎉 ALL DONE!
1. Model is live at: https://huggingface.co/Rajesh17092004/Kaggle-Assistant-Pro-Llama-3.2-3B
2. Zip file is ready: /kaggle/working/finetuned_teacher_final.zip
3. Zip Size: 345.97 MB


In [17]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, LlamaTokenizer
from huggingface_hub import notebook_login
#from datasets import load_dataset
from peft import PeftModel

if "COLAB_GPU" in os.environ:
  from google.colab import output
  output.enable_custom_widget_manager()

In [18]:
if "COLAB_GPU" in os.environ:
  !huggingface-cli login
else:
  notebook_login()

In [6]:
!pip uninstall -y bitsandbytes

Found existing installation: bitsandbytes 0.49.1
Uninstalling bitsandbytes-0.49.1:
  Successfully uninstalled bitsandbytes-0.49.1


In [7]:
pip install -U bitsandbytes

  Using cached bitsandbytes-0.49.1-py3-none-manylinux_2_24_x86_64.whl.metadata (10 kB)
Using cached bitsandbytes-0.49.1-py3-none-manylinux_2_24_x86_64.whl (59.1 MB)
Note: you may need to restart the kernel to use updated packages.


In [8]:
base_model_id = "meta-llama/Llama-3.2-3B-Instruct"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(base_model_id, quantization_config=bnb_config,device_map="auto")

config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

In [9]:
tokenizer = AutoTokenizer.from_pretrained(base_model_id, trust_remote_code=True) # 2. Use AutoTokenizer

if tokenizer.pad_token is None:
  tokenizer.add_special_tokens({'pad_token': tokenizer.eos_token})

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [18]:
tokenized_train_dataset = []
for phrase in kaggle_data:
  tokenized_train_dataset.append(tokenizer(phrase))

In [19]:
def get_dir_size(path='.'):
    """Calculates the size of a directory in megabytes."""
    total_size = 0
    for dirpath, dirnames, filenames in os.walk(path):
        for f in filenames:
            fp = os.path.join(dirpath, f)
            total_size += os.path.getsize(fp)
    return total_size / (1024 * 1024)

In [ ]:
import shutil
import os
from peft import PeftModel

adapter_path = "/kaggle/input/teacher-weights/"
merged_model_save_path = "/kaggle/working/merged_finetuned_model"

print(f"Loading adapter from: {adapter_path}...")

fine_tuned_model = PeftModel.from_pretrained(model, adapter_path)

print("Merging adapter into the base model...")
merged_model = fine_tuned_model.merge_and_unload()
print("Merge complete.")

print(f"Saving merged model to: {merged_model_save_path}...")
merged_model.save_pretrained(merged_model_save_path)
print("Save complete.")

print("Calculating size of the merged model on disk...")

try:
    disk_size_mb = get_dir_size(merged_model_save_path)
    print("---" * 10)
    print(f"Full Merged Model Size: {disk_size_mb:.2f} MB")
    print("---" * 10)
except NameError:
    print("\n[Error] The 'get_dir_size' function is not defined.")
    print("Please run the cell that defines 'get_dir_size' and try again.")
except Exception as e:
    print(f"\nAn error occurred: {e}")

print(f"Cleaning up by deleting: {merged_model_save_path}")
if os.path.exists(merged_model_save_path):
    shutil.rmtree(merged_model_save_path)
    print("Cleanup complete.")
else:
    print("Cleanup not needed, directory does not exist.")

Loading adapter from: /kaggle/input/teacher-weights/...
Merging adapter into the base model...


/usr/local/lib/python3.12/dist-packages/peft/tuners/lora/bnb.py:348: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


Merge complete.
Saving merged model to: /kaggle/working/merged_finetuned_model...
Save complete.
Calculating size of the merged model on disk...
------------------------------
Full Merged Model Size: 2138.87 MB
------------------------------
Cleaning up by deleting: /kaggle/working/merged_finetuned_model
Cleanup complete.


In [ ]:
def measure_latency(model, tokenizer, query="Explain the concept of overfitting. Why is it a problem, and what is one common technique to prevent it?"):
    """Measures the average latency of the model over several runs."""
    messages = [{"role": "user", "content": query}]
    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    
    latencies = []
    print("Running warmup...")
    for _ in range(5):
        _ = model.generate(**inputs, max_new_tokens=50, eos_token_id=tokenizer.eos_token_id)
    
    print("Running timed benchmark...")
    for _ in range(20):
        start_time = time.perf_counter()
        _ = model.generate(**inputs, max_new_tokens=50, eos_token_id=tokenizer.eos_token_id)
        latency = (time.perf_counter() - start_time) * 1000 # Convert to milliseconds
        latencies.append(latency)
        
    avg_latency = np.mean(latencies)
    std_latency = np.std(latencies)
    return {"avg_latency_ms": avg_latency, "std_latency_ms": std_latency}

In [22]:
import torch
import os
import time
import numpy as np

In [23]:
print("TEACHER MODEL LATENCY")

merged_teacher_latency = measure_latency(merged_model, tokenizer)

print(f"MERGED Teacher Latency: {merged_teacher_latency['avg_latency_ms']:.2f} +/- {merged_teacher_latency['std_latency_ms']:.2f} ms")


Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


TEACHER MODEL LATENCY
Running warmup...


Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Running timed benchmark...


Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for

MERGED Teacher Latency: 4232.06 +/- 40.10 ms


In [24]:
user_question = "What is kaggle?"

eval_prompt = f'Question:\n{user_question}\n\Just answer this question accurately and concisely.\n'

promptTokenized = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

fine_tuned_model.eval()

with torch.no_grad():
  print(tokenizer.decode(fine_tuned_model.generate(**promptTokenized, max_new_tokens=256,repetition_penalty=1.2,eos_token_id=tokenizer.eos_token_id)[0], skip_special_tokens=True))
  torch.cuda.empty_cache()

<>:3: SyntaxWarning: invalid escape sequence '\J'
<>:3: SyntaxWarning: invalid escape sequence '\J'
/tmp/ipykernel_55/2090662432.py:3: SyntaxWarning: invalid escape sequence '\J'
  eval_prompt = f'Question:\n{user_question}\n\Just answer this question accurately and concisely.\n'
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Question:
What is kaggle?
\Just answer this question accurately and concisely.
Kaggle is a platform for data science competitions, hosting datasets to be used in such competitions.


In [25]:
user_question = "Write a Python function called find_max that takes a list of numbers as input and returns the largest number in the list."

eval_prompt = f'Question:\n{user_question}\n\Just answer this question accurately and concisely.\n'

promptTokenized = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

fine_tuned_model.eval()

with torch.no_grad():
  print(tokenizer.decode(fine_tuned_model.generate(**promptTokenized, max_new_tokens=256,repetition_penalty=1.2,eos_token_id=tokenizer.eos_token_id)[0], skip_special_tokens=True))
  torch.cuda.empty_cache()

<>:3: SyntaxWarning: invalid escape sequence '\J'
<>:3: SyntaxWarning: invalid escape sequence '\J'
/tmp/ipykernel_55/3007961873.py:3: SyntaxWarning: invalid escape sequence '\J'
  eval_prompt = f'Question:\n{user_question}\n\Just answer this question accurately and concisely.\n'
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Question:
Write a Python function called find_max that takes a list of numbers as input and returns the largest number in the list.
\Just answer this question accurately and concisely.
Answer: 
def find_max(lst):
    return max(lst)


In [26]:
user_question = "Write a Python function called is_even that takes an integer as input and returns True if the number is even and False otherwise"

eval_prompt = f'Question:\n{user_question}\n\Just answer this question accurately and concisely.\n'

promptTokenized = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

fine_tuned_model.eval()

with torch.no_grad():
  print(tokenizer.decode(fine_tuned_model.generate(**promptTokenized, max_new_tokens=256,repetition_penalty=1.2,eos_token_id=tokenizer.eos_token_id)[0], skip_special_tokens=True))
  torch.cuda.empty_cache()

<>:3: SyntaxWarning: invalid escape sequence '\J'
<>:3: SyntaxWarning: invalid escape sequence '\J'
/tmp/ipykernel_55/3710983772.py:3: SyntaxWarning: invalid escape sequence '\J'
  eval_prompt = f'Question:\n{user_question}\n\Just answer this question accurately and concisely.\n'
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Question:
Write a Python function called is_even that takes an integer as input and returns True if the number is even and False otherwise
\Just answer this question accurately and concisely.
Answer: 
def is_even(n):
    return n % 2 == 0


In [27]:
user_question = "What is the difference between classification and regression in machine learning? Can you give an example of each?"

eval_prompt = f'Question:\n{user_question}\n\Just answer this question accurately and concisely.\n'

promptTokenized = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

fine_tuned_model.eval()

with torch.no_grad():
  print(tokenizer.decode(fine_tuned_model.generate(**promptTokenized, max_new_tokens=256,repetition_penalty=1.2,eos_token_id=tokenizer.eos_token_id)[0], skip_special_tokens=True))
  torch.cuda.empty_cache()

<>:3: SyntaxWarning: invalid escape sequence '\J'
<>:3: SyntaxWarning: invalid escape sequence '\J'
/tmp/ipykernel_55/4058350022.py:3: SyntaxWarning: invalid escape sequence '\J'
  eval_prompt = f'Question:\n{user_question}\n\Just answer this question accurately and concisely.\n'
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Question:
What is the difference between classification and regression in machine learning? Can you give an example of each?
\Just answer this question accurately and concisely.
Classification: Assigns a categorical label to input data. Regression: Predicts continuous values for input variables.

Example Classification: Breast Cancer Diagnosis - A model predicts whether breast cancer is malignant or benign based on certain features (e.g., tumor size, cell count). Example Regression: House Prices Prediction - A model predicts house prices based on factors like location, number of bedrooms, etc.


In [28]:
user_question = "Explain the concept of overfitting. Why is it a problem, and what is one common technique to prevent it?"

eval_prompt = f'Question:\n{user_question}\n\Just answer this question accurately and concisely.\n'

promptTokenized = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

fine_tuned_model.eval()

with torch.no_grad():
  print(tokenizer.decode(fine_tuned_model.generate(**promptTokenized, max_new_tokens=100,repetition_penalty=1.2,eos_token_id=tokenizer.eos_token_id)[0], skip_special_tokens=True))
  torch.cuda.empty_cache()

<>:3: SyntaxWarning: invalid escape sequence '\J'
<>:3: SyntaxWarning: invalid escape sequence '\J'
/tmp/ipykernel_55/890702873.py:3: SyntaxWarning: invalid escape sequence '\J'
  eval_prompt = f'Question:\n{user_question}\n\Just answer this question accurately and concisely.\n'
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Question:
Explain the concept of overfitting. Why is it a problem, and what is one common technique to prevent it?
\Just answer this question accurately and concisely.
Overfitting occurs when a model becomes too specialized in fitting the noise present in training data rather than generalizing well to unseen or test data. This can happen if we have more parameters than necessary for our dataset size, leading the model to memorize specific patterns instead of learning meaningful relationships between variables.

One common technique to prevent overfitting is regularization, which adds a penalty term to the loss function that discourages large weights by increasing the cost of complex models. For example, L1 (L


In [29]:
user_question = "What is a p-value, and what does it typically signify in the context of hypothesis testing?"

eval_prompt = f'Question:\n{user_question}\n\Just answer this question accurately and concisely.\n'

promptTokenized = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

fine_tuned_model.eval()

with torch.no_grad():
  print(tokenizer.decode(fine_tuned_model.generate(**promptTokenized, max_new_tokens=256,repetition_penalty=1.2,eos_token_id=tokenizer.eos_token_id)[0], skip_special_tokens=True))
  torch.cuda.empty_cache()

<>:3: SyntaxWarning: invalid escape sequence '\J'
<>:3: SyntaxWarning: invalid escape sequence '\J'
/tmp/ipykernel_55/270570122.py:3: SyntaxWarning: invalid escape sequence '\J'
  eval_prompt = f'Question:\n{user_question}\n\Just answer this question accurately and concisely.\n'
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Question:
What is a p-value, and what does it typically signify in the context of hypothesis testing?
\Just answer this question accurately and concisely.
Answer:  A p-value represents the probability of observing an effect at least as extreme as that observed during an experiment or study, assuming no real effect exists. It signifies whether there's sufficient evidence to reject a null hypothesis based on statistical significance levels (usually α = 0.05).


In [30]:
user_question = "how can i become a grandmaster in kaggle?"

eval_prompt = f'Question:\n{user_question}\n\Just answer this question accurately and concisely.\n'

promptTokenized = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

fine_tuned_model.eval()

with torch.no_grad():
  print(tokenizer.decode(fine_tuned_model.generate(**promptTokenized, max_new_tokens=256,repetition_penalty=1.2,eos_token_id=tokenizer.eos_token_id)[0], skip_special_tokens=True))
  torch.cuda.empty_cache()

<>:3: SyntaxWarning: invalid escape sequence '\J'
<>:3: SyntaxWarning: invalid escape sequence '\J'
/tmp/ipykernel_55/1962209463.py:3: SyntaxWarning: invalid escape sequence '\J'
  eval_prompt = f'Question:\n{user_question}\n\Just answer this question accurately and concisely.\n'
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Question:
how can i become a grandmaster in kaggle?
\Just answer this question accurately and concisely.
To become a Grand Master on Kaggle, you need to achieve the top rank (100%) across all competitions. Here are some steps that could help:

*   **Participate consistently**: Regularly participate in competitions to gain experience and improve your skills.
*   **Focus on accuracy**: Aim for high accuracy scores by carefully selecting models, tuning hyperparameters, and testing different approaches.
*   **Stay up-to-date with trends**: Keep track of emerging trends and techniques in machine learning to stay ahead of the competition.

Becoming a Grand Master takes time, dedication, and persistence. Focus on continuous improvement and strive to excel in each competition. Good luck!


DISTILLATION

In [31]:
base_model_id = "meta-llama/Llama-3.2-3B-Instruct"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

techer_model = AutoModelForCausalLM.from_pretrained(base_model_id, quantization_config=bnb_config,device_map="auto")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(base_model_id, trust_remote_code=True) 

if tokenizer.pad_token is None:
  tokenizer.add_special_tokens({'pad_token': tokenizer.eos_token})

In [33]:
from peft import PeftModel
teacher_model = PeftModel.from_pretrained(techer_model,"/kaggle/input/teacher-weights/")

In [34]:
for param in teacher_model.parameters():
    param.requires_grad = False
teacher_model.eval()
print("Teacher model loaded and frozen.")


Teacher model loaded and frozen.


In [35]:
student_model_id = "meta-llama/Llama-3.2-1B-Instruct"
student_model = AutoModelForCausalLM.from_pretrained(student_model_id, quantization_config=bnb_config,device_map="auto")

config.json:   0%|          | 0.00/877 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

In [36]:
from peft import get_peft_model,LoraConfig,prepare_model_for_kbit_training

In [37]:
student_model.gradient_checkpointing_enable()
student_model = prepare_model_for_kbit_training(student_model)

config = LoraConfig(
    r=64,
    lora_alpha=64,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    bias="none",
    lora_dropout=0.05,
    task_type="CAUSAL_LM"
)
student_model = get_peft_model(student_model, config)

In [ ]:
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling

class DistillationTrainer(Trainer):
    def __init__(self, *args, teacher_model=None, **kwargs):
        super().__init__(*args, **kwargs)
        self.teacher_model = teacher_model

    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        outputs_stu = model(**inputs)
        logits_stu = outputs_stu.logits
        with torch.no_grad():
            outputs_tea = self.teacher_model(**inputs)
            logits_tea = outputs_tea.logits
        loss_fct = nn.KLDivLoss(reduction="batchmean")
        loss_kd = self.args.temperature ** 2 * loss_fct(
            F.log_softmax(logits_stu / self.args.temperature, dim=-1),
            F.softmax(logits_tea / self.args.temperature, dim=-1)
        )

        return (loss_kd, outputs_stu) if return_outputs else loss_kd

class DistillationTrainingArguments(TrainingArguments):
    def __init__(self, *args, temperature=2.0, **kwargs):
        super().__init__(*args, **kwargs)
        self.temperature = temperature

training_args = DistillationTrainingArguments(
    output_dir="./distilled_model_checkpoints",
    num_train_epochs=3,                 
    per_device_train_batch_size=1,      
    gradient_accumulation_steps=16,
    fp16=True,
    learning_rate=5e-5,
    save_strategy="epoch",
    logging_steps=50,
    temperature=2.0,                    
    report_to="none",
)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, 
    mlm=False
)

distillation_trainer = DistillationTrainer(
    model=student_model,              
    teacher_model=teacher_model,       
    args=training_args,
    train_dataset=tokenized_train_dataset,    
    data_collator=data_collator,
    tokenizer=tokenizer,
)

print(" DistillationTrainer is configured and ready for training.")
distillation_trainer.train()
print("✅Distillation training has completed successfully!")
torch.cuda.empty_cache()

/tmp/ipykernel_55/1249504526.py:7: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `DistillationTrainer.__init__`. Use `processing_class` instead.
  super().__init__(*args, **kwargs)
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'eos_token_id': 128009, 'pad_token_id': 128009}.


✅ DistillationTrainer is configured and ready for training.


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:929: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
50,7184.609400
100,5316.736900
150,4827.927500
200,4303.109700
250,4204.884400
300,4094.101900
350,4217.531600


/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:929: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:929: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


✅ Distillation training has completed successfully!


In [ ]:
import os, glob, shutil
from pathlib import Path

OUT_DIR = Path("./distilled_model_checkpoints")

ckpts = sorted([p for p in OUT_DIR.glob("**/*") if p.is_dir() and ("checkpoint" in p.name or p.name=="finetunedModel" or p.name=="adapter")])
print("Found candidate dirs (last few):")
for x in ckpts[-10:]:
    print(" ", x)

checkpoint_dirs = sorted([d for d in OUT_DIR.iterdir() if d.is_dir() and ("checkpoint" in d.name or d.name=="adapter")])
if len(checkpoint_dirs) > 0:
    latest_ckpt = checkpoint_dirs[-1]
else:
    latest_ckpt = OUT_DIR 

print("Using checkpoint folder:", latest_ckpt)
zip_name = "/kaggle/working/distilled_model_final"
shutil.make_archive(zip_name, 'zip', root_dir=str(latest_ckpt))
print("Zipped to:", zip_name + ".zip")
print("Size (bytes):", os.path.getsize(zip_name + ".zip"))


Found candidate dirs (last few):
  distilled_model_checkpoints/checkpoint-123
  distilled_model_checkpoints/checkpoint-246
  distilled_model_checkpoints/checkpoint-369
Using checkpoint folder: distilled_model_checkpoints/checkpoint-369
Zipped to: /kaggle/working/distilled_model_final.zip
Size (bytes): 498554212


In [41]:
distilled_model_id = "meta-llama/Llama-3.2-1B-Instruct"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

distilled_model = AutoModelForCausalLM.from_pretrained(distilled_model_id, quantization_config=bnb_config,device_map="auto")

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(distilled_model_id, trust_remote_code=True)

if tokenizer.pad_token is None:
  tokenizer.add_special_tokens({'pad_token': tokenizer.eos_token})
from peft import PeftModel
distilled_model = PeftModel.from_pretrained(distilled_model,"distilled_model_checkpoints/checkpoint-369")

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [45]:
user_question = "how can i become a grandmaster in kaggle?"

eval_prompt = f'Question:\n{user_question}\n\Just answer this question accurately and concisely.\n'

promptTokenized = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

distilled_model.eval()

with torch.no_grad():
  print(tokenizer.decode(distilled_model.generate(**promptTokenized, max_new_tokens=256,repetition_penalty=1.2,eos_token_id=tokenizer.eos_token_id)[0], skip_special_tokens=True))
  torch.cuda.empty_cache()

<>:3: SyntaxWarning: invalid escape sequence '\J'
<>:3: SyntaxWarning: invalid escape sequence '\J'
/tmp/ipykernel_55/1373154909.py:3: SyntaxWarning: invalid escape sequence '\J'
  eval_prompt = f'Question:\n{user_question}\n\Just answer this question accurately and concisely.\n'
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Question:
how can i become a grandmaster in kaggle?
\Just answer this question accurately and concisely.
\Answer:
To be a Grand Master on Kaggle, you need to have at least 1000 points. You can achieve that by participating in the competition with your best performance or by getting high scores in specific competitions. If you want more details, I'd recommend checking out the official Kaggle website for more information. (Source: https://www.kaggle.com/ )


In [46]:
user_question = "Write a Python function called find_max that takes a list of numbers as input and returns the largest number in the list."

eval_prompt = f'Question:\n{user_question}\n\Just answer this question accurately and concisely.\n'

promptTokenized = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

distilled_model.eval()

with torch.no_grad():
  print(tokenizer.decode(distilled_model.generate(**promptTokenized, max_new_tokens=256,repetition_penalty=1.2,eos_token_id=tokenizer.eos_token_id)[0], skip_special_tokens=True))
  torch.cuda.empty_cache()

<>:3: SyntaxWarning: invalid escape sequence '\J'
<>:3: SyntaxWarning: invalid escape sequence '\J'
/tmp/ipykernel_55/2678165030.py:3: SyntaxWarning: invalid escape sequence '\J'
  eval_prompt = f'Question:\n{user_question}\n\Just answer this question accurately and concisely.\n'
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Question:
Write a Python function called find_max that takes a list of numbers as input and returns the largest number in the list.
\Just answer this question accurately and concisely.
\Answer:
def find_max(list1): 
    max = max(list1) 
    return max  # Return the maximum value from the list. ​
    # If you want to get all values, use list(max(list1)) ​
    # or use list(map(max, list1)) ​
    # or use list(sorted(list1)) ​
    # Or just use max(list1) if you have only one element in your list. ​
    # For example: print(find_max([3,2,4,5,6]))  # Output: 6 ``` This is a simple solution using the built-in max() function which returns the highest value in the given list. You can also write it yourself by using the following code: def find_max(list1): 
    max = list1[0] 
    for i in range(1, len(list1)): 
        if list1[i] > max: 
            max = list1[i] 
    return max  # Return the maximum value from the list. ​

# Test the function with an example: print(find_max([3,2,4,5,6]) 

In [54]:
user_question = "Write a Python function called is_even that takes an integer as input and returns True if the number is even and False otherwise."

eval_prompt = f'Question:\n{user_question}\n\Just answer this question accurately and concisely.\n'

promptTokenized = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

distilled_model.eval()

with torch.no_grad():
  print(tokenizer.decode(distilled_model.generate(**promptTokenized, max_new_tokens=256,repetition_penalty=1.2,eos_token_id=tokenizer.eos_token_id)[0], skip_special_tokens=True))
  torch.cuda.empty_cache()

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Question:
Write a Python function called is_even that takes an integer as input and returns True if the number is even and False otherwise.
\Just answer this question accurately and concisely.
\Answer:
def is_even(n): 
    return n % 2 == 0;  # Returns true if n is even, false else.  # Using lambda function for simplicity.  def is_even(n): return (lambda x: x % 2 == 0) (n)  # The above code can be written in one line using lambda function.  # Example use case: print(is_even(10))  # Output: True  # Example use case: print(is_even(-1))  # Output: False  # Example use case: print(is_even(3))  # Output: True  # Example use case: print(is_even(4))  # Output: True  # Example use case: print(is_even(5))  # Output: False  # Example use case: print(is_even(6))  # Output: True  # Example use case: print(is_even(7))  # Output: False  # Example use case: print(is_even(8))  # Output: True  # Example use case: print(is_even(9))  # Output: False  # Example use case: print(is_even(10))  # Output: True

In [47]:
user_question = "what is the difference between the classification and regression in machine learning?"

eval_prompt = f'Question:\n{user_question}\n\Just answer this question accurately and concisely.\n'

promptTokenized = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

distilled_model.eval()

with torch.no_grad():
  print(tokenizer.decode(distilled_model.generate(**promptTokenized, max_new_tokens=256,repetition_penalty=1.2,eos_token_id=tokenizer.eos_token_id)[0], skip_special_tokens=True))
  torch.cuda.empty_cache()

<>:3: SyntaxWarning: invalid escape sequence '\J'
<>:3: SyntaxWarning: invalid escape sequence '\J'
/tmp/ipykernel_55/1021915395.py:3: SyntaxWarning: invalid escape sequence '\J'
  eval_prompt = f'Question:\n{user_question}\n\Just answer this question accurately and concisely.\n'
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Question:
what is the difference between the classification and regression in machine learning?
\Just answer this question accurately and concisely.
\Answer:
Classification problems are those where you have a set of input data and you want to predict which class or category that data belongs to. Regression problems, on the other hand, are those where you want to predict a continuous value (e.g., price of an item). In general, if you're trying to classify something into one of several categories, you'll use a classifier. If you're trying to forecast a continuous value, you'll use a regressor.


In [48]:
user_question = "Explain the concept of overfitting. Why is it a problem, and what is one common technique to prevent it?"

eval_prompt = f'Question:\n{user_question}\n\Just answer this question accurately and concisely.\n'

promptTokenized = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

distilled_model.eval()

with torch.no_grad():
  print(tokenizer.decode(distilled_model.generate(**promptTokenized, max_new_tokens=256,repetition_penalty=1.2,eos_token_id=tokenizer.eos_token_id)[0], skip_special_tokens=True))
  torch.cuda.empty_cache()

<>:3: SyntaxWarning: invalid escape sequence '\J'
<>:3: SyntaxWarning: invalid escape sequence '\J'
/tmp/ipykernel_55/181090311.py:3: SyntaxWarning: invalid escape sequence '\J'
  eval_prompt = f'Question:\n{user_question}\n\Just answer this question accurately and concisely.\n'
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Question:
Explain the concept of overfitting. Why is it a problem, and what is one common technique to prevent it?
\Just answer this question accurately and concisely.
\Answer:
Overfitting occurs when a model becomes too specialized in the training data and fails to generalize well to new, unseen data. This can happen if the model has too many parameters or if there are too many features in the dataset. One common technique to prevent overfitting is to use regularization techniques such as L1 or L2 regularization, which add penalty terms to the loss function to discourage large parameter values. Another approach is to use ensemble methods, which combine the predictions of multiple models to improve generalizability. Finally, using more data or increasing the size of the training set can help reduce overfitting.


In [ ]:
import shutil
import os
import torch
from transformers import AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel

student_base_model_id = "meta-llama/Llama-3.2-1B-Instruct"
student_adapter_path = "distilled_model_checkpoints/checkpoint-369"
merged_student_save_path = "/kaggle/working/merged_distilled_model"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

print(f"Loading base student model: {student_base_model_id}...")
base_student_model = AutoModelForCausalLM.from_pretrained(
    student_base_model_id,
    quantization_config=bnb_config,
    device_map="auto"
)

print(f"Loading student adapter from: {student_adapter_path}...")
distilled_peft_model = PeftModel.from_pretrained(base_student_model, student_adapter_path)

print("Merging student adapter into the base model...")
merged_student_model = distilled_peft_model.merge_and_unload()
print("Merge complete.")
print(f"Saving merged student model to: {merged_student_save_path}...")
merged_student_model.save_pretrained(merged_student_save_path)
print("Save complete.")
print("Calculating size of the merged student model on disk...")
try:
    student_disk_size_mb = get_dir_size(merged_student_save_path)
    print("---" * 10)
    print(f"Full Merged DISTILLED STUDENT Model Size: {student_disk_size_mb:.2f} MB")
    print("---" * 10)
except NameError:
    print("\n[Error] The 'get_dir_size' function is not defined.")
    print("Please run the cell that defines 'get_dir_size' and try again.")
except Exception as e:
    print(f"\nAn error occurred: {e}")
print(f"Cleaning up by deleting: {merged_student_save_path}")
if os.path.exists(merged_student_save_path):
    shutil.rmtree(merged_student_save_path)
    print("Cleanup complete.")
else:
    print("Cleanup not needed, directory does not exist.")

Loading base student model: meta-llama/Llama-3.2-1B-Instruct...
Loading student adapter from: distilled_model_checkpoints/checkpoint-369...
Merging student adapter into the base model...


/usr/local/lib/python3.12/dist-packages/peft/tuners/lora/bnb.py:348: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


Merge complete.
Saving merged student model to: /kaggle/working/merged_distilled_model...
Save complete.
Calculating size of the merged student model on disk...
------------------------------
Full Merged DISTILLED STUDENT Model Size: 980.07 MB
------------------------------
Cleaning up by deleting: /kaggle/working/merged_distilled_model
Cleanup complete.


In [50]:
print("STUDENT MODEL LATENCY")
merged_student_latency = measure_latency(merged_student_model, tokenizer)
print(f"MERGED Student Latency: {merged_student_latency['avg_latency_ms']:.2f} +/- {merged_student_latency['std_latency_ms']:.2f} ms")

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


STUDENT MODEL LATENCY
Running warmup...


Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Running timed benchmark...


Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for

MERGED Student Latency: 2290.54 +/- 24.96 ms


In [ ]:
MERGED Teacher Latency: 4232.06 +/- 40.10 ms
MERGED Student Latency: 2290.54 +/- 24.96 ms

Full Merged Model Size: 2138.87 MB 
Full Merged DISTILLED STUDENT Model Size: 980.07 MB